In [1]:
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
m = pd.read_csv("../../data/raw/KOBIS_개봉일람.csv", encoding="cp949")

m

,순번,영화명,감독,제작사,수입사,배급사,개봉일,영화유형,영화형태,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,등급,영화구분
0,1,명량,김한민,(주)빅스톤픽쳐스,NaN,(주)씨제이이엔엠,2014-07-30,개봉영화,장편,한국,1587,1.357484e+11,17613682,3.312123e+10,4163666,사극,15세이상관람가,일반영화
1,2,극한직업,이병헌,(주)어바웃잇영화사 해그림 주식회사(주)씨제이이엔엠,NaN,(주)씨제이이엔엠,2019-01-23,개봉영화,장편,한국,1978,1.396480e+11,16264944,3.185866e+10,3638287,코미디,15세이상관람가,일반영화
2,3,신과함께-죄와 벌,김용화,리얼라이즈픽쳐스(주)(주)덱스터스튜디오,NaN,롯데쇼핑㈜롯데엔터테인먼트,2017-12-20,개봉영화,장편,한국,1912,1.156987e+11,14410754,2.753083e+10,3346172,판타지,12세이상관람가,일반영화
3,4,국제시장,윤제균,(주)제이케이필름(주)씨제이이엔엠,NaN,(주)씨제이이엔엠,2014-12-17,개봉영화,장편,한국,966,1.108280e+11,14245998,2.584252e+10,3233946,드라마,12세이상관람가,일반영화
4,5,어벤져스: 엔드게임,안소니 루소조 루소,NaN,월트디즈니컴퍼니코리아 유한책임회사,월트디즈니컴퍼니코리아 유한책임회사,2019-04-24,개봉영화,장편,미국,2835,1.221827e+11,13934592,3.357714e+10,3597963,액션,12세이상관람가,일반영화
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27399,27400,V2 폭파대작전,엔조 G. 카스텔라리,NaN,(주)동아수출공사,NaN,1978-12-21,개봉영화,장편,이탈리아,0,NaN,0,NaN,174276,액션,NaN,일반영화
27400,27401,W의 비극,김수형,한진흥업주식회사,NaN,NaN,1985-06-09,개봉영화,장편,한국,0,0.000000e+00,0,0.000000e+00,8624,드라마,연소자관람불가,일반영화
27401,27402,X게임,엠마뉴엘 그리센티,NaN,(주)한영필림,NaN,1997-07-05,개봉영화,장편,이탈리아,0,0.000000e+00,0,0.000000e+00,434,범죄,연소자관람불가,일반영화
27402,27403,Y의 체험,이장호,판영화사(주),NaN,NaN,1987-10-03,개봉영화,장편,한국,0,0.000000e+00,0,0.000000e+00,37520,성인물(에로),고등학생이상관람가,일반영화


In [3]:
# 순번, 배급사, 수입사 컬럼 삭제
m = m.drop(["순번", "배급사", "수입사"], axis=1)

In [4]:
m.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27404 entries, 0 to 27403
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   영화명      27404 non-null  object 
 1   감독       25745 non-null  object 
 2   제작사      11666 non-null  object 
 3   개봉일      27382 non-null  object 
 4   영화유형     27404 non-null  object 
 5   영화형태     27387 non-null  object 
 6   국적       27393 non-null  object 
 7   전국 스크린수  27404 non-null  int64  
 8   전국 매출액   26348 non-null  float64
 9   전국 관객수   27404 non-null  int64  
 10  서울 매출액   26342 non-null  float64
 11  서울 관객수   27404 non-null  int64  
 12  장르       27213 non-null  object 
 13  등급       26916 non-null  object 
 14  영화구분     27404 non-null  object 
dtypes: float64(2), int64(3), object(10)
memory usage: 3.1+ MB


In [5]:
m.isna().sum()

영화명            0
감독          1659
제작사        15738
개봉일           22
영화유형           0
영화형태          17
국적            11
전국 스크린수        0
전국 매출액      1056
전국 관객수         0
서울 매출액      1062
서울 관객수         0
장르           191
등급           488
영화구분           0
dtype: int64

In [6]:
# 전국 스크린수 컬럼에서 평균보다 큰 데이터만 선택
m = m[m["전국 스크린수"] > m["전국 스크린수"].mean()]

m

,영화명,감독,제작사,개봉일,영화유형,영화형태,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,등급,영화구분
0,명량,김한민,(주)빅스톤픽쳐스,2014-07-30,개봉영화,장편,한국,1587,1.357484e+11,17613682,3.312123e+10,4163666,사극,15세이상관람가,일반영화
1,극한직업,이병헌,(주)어바웃잇영화사 해그림 주식회사(주)씨제이이엔엠,2019-01-23,개봉영화,장편,한국,1978,1.396480e+11,16264944,3.185866e+10,3638287,코미디,15세이상관람가,일반영화
2,신과함께-죄와 벌,김용화,리얼라이즈픽쳐스(주)(주)덱스터스튜디오,2017-12-20,개봉영화,장편,한국,1912,1.156987e+11,14410754,2.753083e+10,3346172,판타지,12세이상관람가,일반영화
3,국제시장,윤제균,(주)제이케이필름(주)씨제이이엔엠,2014-12-17,개봉영화,장편,한국,966,1.108280e+11,14245998,2.584252e+10,3233946,드라마,12세이상관람가,일반영화
4,어벤져스: 엔드게임,안소니 루소조 루소,NaN,2019-04-24,개봉영화,장편,미국,2835,1.221827e+11,13934592,3.357714e+10,3597963,액션,12세이상관람가,일반영화
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8041,늑대와 빨간 재킷,뱅상 파로노,NaN,2021-02-03,개봉영화,장편,프랑스,74,8.525200e+06,950,2.893740e+06,345,스릴러,청소년관람불가,독립/예술영화
8157,마라가 큰 결정을 해야 해,로버트 루케틱,NaN,2021-01-07,개봉영화,장편,미국,79,7.475940e+06,856,2.830800e+06,321,멜로/로맨스,15세이상관람가,일반영화
8165,락다운 213주,아담 메이슨,NaN,2022-08-31,개봉영화,장편,미국,61,7.664799e+06,845,3.573900e+06,381,스릴러,15세이상관람가,일반영화
8263,래미의 드래곤월드 구출작전,웨이밍 황,NaN,2020-07-16,개봉영화,장편,중국,65,6.067340e+06,779,6.345000e+05,74,애니메이션,전체관람가,일반영화


In [7]:
m.isna().sum()

영화명           0
감독           99
제작사        2541
개봉일           0
영화유형          0
영화형태          0
국적            0
전국 스크린수       0
전국 매출액        0
전국 관객수        0
서울 매출액        0
서울 관객수        0
장르            0
등급            0
영화구분          0
dtype: int64

In [8]:
m.loc[m["감독"].isna()]

,영화명,감독,제작사,개봉일,영화유형,영화형태,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,등급,영화구분
1608,러브 유어셀프 인 서울,NaN,빅히트 엔터테인먼트씨제이 씨지브이(CJ CGV)(주),2019-01-26,개봉영화,장편,한국,224,3.209683e+09,342366,1.090468e+09,112697,공연,전체관람가,일반영화
1966,프리즌 이스케이프,NaN,NaN,2020-05-06,개봉영화,장편,영국,328,1.848298e+09,217056,5.564777e+08,63837,어드벤처,12세이상관람가,독립/예술영화
2473,로스트 시티,NaN,NaN,2022-04-20,개봉영화,장편,미국,835,1.023085e+09,113186,3.581225e+08,41462,액션,12세이상관람가,일반영화
2567,그대 고맙소 : 김호중 생애 첫 팬미팅 무비,NaN,씨제이포디플렉스 주식회사,2020-09-29,개봉영화,장편,한국,106,3.129974e+09,102008,9.867305e+08,31301,공연,전체관람가,일반영화
2732,던전 앤 드래곤: 도적들의 명예,NaN,NaN,2023-03-29,개봉영화,장편,미국,1105,7.942647e+08,83761,2.781635e+08,27446,액션,12세관람가,일반영화
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7594,코드 8,NaN,NaN,2020-07-02,개봉영화,장편,캐나다,82,1.124136e+07,1362,3.337960e+06,400,SF,15세이상관람가,일반영화
7665,섀도우 클라우드,NaN,NaN,2021-04-22,개봉영화,장편,미국,71,1.151214e+07,1282,3.775040e+06,384,SF,15세이상관람가,일반영화
7679,엠티맨,NaN,NaN,2020-11-19,개봉영화,장편,미국,100,1.131412e+07,1266,4.967810e+06,556,공포(호러),15세이상관람가,일반영화
7822,퍼펙트 스틸,NaN,NaN,2021-10-28,개봉영화,장편,영국,66,1.007310e+07,1122,5.621200e+06,604,범죄,15세이상관람가,일반영화


In [9]:
m.loc[m["제작사"].isna()]

,영화명,감독,제작사,개봉일,영화유형,영화형태,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,등급,영화구분
4,어벤져스: 엔드게임,안소니 루소조 루소,NaN,2019-04-24,개봉영화,장편,미국,2835,1.221827e+11,13934592,3.357714e+10,3597963,액션,12세이상관람가,일반영화
5,겨울왕국 2,크리스 벅제니퍼 리,NaN,2019-11-21,개봉영화,장편,미국,2648,1.148104e+11,13747792,2.882468e+10,3225974,애니메이션,전체관람가,일반영화
13,알라딘,가이 리치,NaN,2019-05-23,개봉영화,장편,미국,1311,1.069836e+11,12555894,3.021891e+10,3377404,어드벤처,전체관람가,일반영화
22,어벤져스: 인피니티 워,안소니 루소조 루소,NaN,2018-04-25,개봉영화,장편,미국,2553,9.991901e+10,11211880,2.863127e+10,3008819,액션,12세이상관람가,일반영화
24,아바타: 물의 길,제임스 카메론,NaN,2022-12-14,개봉영화,장편,미국,2809,1.375873e+11,10801074,3.545201e+10,2621620,액션,12세이상관람가,일반영화
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8041,늑대와 빨간 재킷,뱅상 파로노,NaN,2021-02-03,개봉영화,장편,프랑스,74,8.525200e+06,950,2.893740e+06,345,스릴러,청소년관람불가,독립/예술영화
8157,마라가 큰 결정을 해야 해,로버트 루케틱,NaN,2021-01-07,개봉영화,장편,미국,79,7.475940e+06,856,2.830800e+06,321,멜로/로맨스,15세이상관람가,일반영화
8165,락다운 213주,아담 메이슨,NaN,2022-08-31,개봉영화,장편,미국,61,7.664799e+06,845,3.573900e+06,381,스릴러,15세이상관람가,일반영화
8263,래미의 드래곤월드 구출작전,웨이밍 황,NaN,2020-07-16,개봉영화,장편,중국,65,6.067340e+06,779,6.345000e+05,74,애니메이션,전체관람가,일반영화


In [10]:
# 개봉일의 형식이 일치하는지 확인
for i, j in enumerate(m.개봉일):
    if len(j.split("-")) != 3:
        print(i)

In [11]:
m.영화유형.unique()

array(['개봉영화'], dtype=object)

In [12]:
m = m.drop(["영화유형"], axis=1)

In [13]:
m.영화형태.unique()

array(['장편', '옴니버스', '단편'], dtype=object)

In [14]:
m.국적.unique()

array(['한국', '미국', '일본', '영국', '중국', '프랑스', '벨기에', '독일', '인도', '아이슬란드',
       '캐나다', '스페인', '러시아', '아일랜드', '남아프리카공화국', '대만', '핀란드', '홍콩', '스웨덴',
       '이탈리아', '호주', '덴마크', '기타', '아르헨티나', '헝가리', '태국', '페루', '멕시코',
       '우크라이나', '노르웨이', '터키', '뉴질랜드', '체코', '그리스', '폴란드', '칠레', '말레이시아',
       '인도네시아', '브라질', '네덜란드', '이스라엘', '에스토니아'], dtype=object)

In [15]:
# 국적 컬럼의 빈도수가 높은 상위 10개 값 선택
nation = "|".join(m["국적"].value_counts().head(10).index)

nation

'미국|한국|일본|영국|프랑스|중국|독일|스페인|러시아|홍콩'

In [16]:
# 상위 10개 값이 아닌 값의 수
print(len(m.loc[~m["국적"].str.contains(nation)]))

# 상위 10개 값이 아닌 값을 기타로 변경
m.loc[~m["국적"].str.contains(nation), "국적"] = "기타"

# 변경 후 기타의 수
print(len(m[m["국적"] == "기타"]))
# 변경 후 고윳값
print(m.국적.unique())

264
264
['한국' '미국' '일본' '영국' '중국' '프랑스' '기타' '독일' '스페인' '러시아' '홍콩']


In [17]:
# 다중공선성을 계산하는 함수
def calculate_vif(data):
    vif = pd.DataFrame()  # 결과를 저장할 빈 데이터프레임 생성
    vif["feature"] = data.columns  # feature 열에 변수 이름 저장
    vif["VIF"] = [variance_inflation_factor(data.values, i) for i in range(data.shape[1])]  # VIF 계산하여 VIF 열에 저장
    return vif

In [18]:
vif_data = m[["전국 스크린수", "전국 매출액", "전국 관객수", "서울 매출액", "서울 관객수"]]
calculate_vif(vif_data)

,feature,VIF
0,전국 스크린수,2.085988
1,전국 매출액,1572.217190
2,전국 관객수,1654.982421
3,서울 매출액,1471.364523
4,서울 관객수,1550.256224


In [19]:
m.장르.unique()

array(['사극', '코미디', '판타지', '드라마', '액션', '애니메이션', 'SF', '범죄', '어드벤처', '전쟁',
       '미스터리', '서부극(웨스턴)', '멜로/로맨스', '스릴러', '다큐멘터리', '공포(호러)', '뮤지컬',
       '가족', '기타', '공연', '성인물(에로)'], dtype=object)

In [20]:
m.등급.unique()
# 전체 관람가, 12세 이상 관람가, 15세 이상 관람가, 청소년 관람 불가, 제한 상영가

array(['15세이상관람가', '12세이상관람가', '전체관람가', '청소년관람불가', '청소년관람불가15세이상관람가',
       '12세관람가', '15세관람가', '12세이상관람가15세이상관람가'], dtype=object)

In [21]:
m[m["등급"] == "청소년관람불가15세이상관람가"]
# 그랜드 부다페스트 호텔 15세
# 그녀 15세

,영화명,감독,제작사,개봉일,영화형태,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,등급,영화구분
1025,그랜드 부다페스트 호텔,웨스 앤더슨,NaN,2014-03-20,장편,미국,163,6.012396e+09,773649,3.516636e+09,444985,미스터리,청소년관람불가15세이상관람가,독립/예술영화
1589,그녀,스파이크 존즈,NaN,2014-05-22,장편,미국,168,2.861230e+09,351091,1.876009e+09,227358,드라마,청소년관람불가15세이상관람가,독립/예술영화


In [22]:
# 검색 결과를 기반으로 값 변경
m.loc[m["등급"] == "청소년관람불가15세이상관람가", "등급"] = "15세이상관람가"

m[m["등급"] == "청소년관람불가15세이상관람가"]

,영화명,감독,제작사,개봉일,영화형태,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,등급,영화구분


In [23]:
m[m["등급"] == "12세관람가"]
# 던전 앤 드래곤: 도적들의 명예 12세

,영화명,감독,제작사,개봉일,영화형태,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,등급,영화구분
2732,던전 앤 드래곤: 도적들의 명예,NaN,NaN,2023-03-29,장편,미국,1105,794264727.0,83761,278163526.0,27446,액션,12세관람가,일반영화


In [24]:
# 검색 결과를 기반으로 값 변경
m.loc[m["등급"] == "12세관람가", "등급"] = "12세이상관람가"

m[m["등급"] == "12세관람가"]

,영화명,감독,제작사,개봉일,영화형태,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,등급,영화구분


In [25]:
m[m["등급"] == "15세관람가"]
# 피아니스트의 전설 15세

,영화명,감독,제작사,개봉일,영화형태,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,등급,영화구분
3349,피아니스트의 전설,쥬세페 토르나토레,NaN,2020-01-01,장편,기타,69,340627562.0,41680,207078442.0,24197,드라마,15세관람가,독립/예술영화


In [26]:
# 검색 결과를 기반으로 값 변경
m.loc[m["등급"] == "15세관람가", "등급"] = "15세이상관람가"

m[m["등급"] == "15세관람가"]

,영화명,감독,제작사,개봉일,영화형태,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,등급,영화구분


In [27]:
m[m["등급"] == "12세이상관람가15세이상관람가"]
# 라이크 크레이지 12세

,영화명,감독,제작사,개봉일,영화형태,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,등급,영화구분
4773,라이크 크레이지,드레이크 도레무스,NaN,2018-05-30,장편,미국,116,81549980.0,10850,39981980.0,5165,드라마,12세이상관람가15세이상관람가,독립/예술영화


In [28]:
# 검색 결과를 기반으로 값 변경
m.loc[m["등급"] == "12세이상관람가15세이상관람가", "등급"] = "12세이상관람가"

m[m["등급"] == "12세이상관람가15세이상관람가"]

,영화명,감독,제작사,개봉일,영화형태,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,등급,영화구분


In [29]:
m.영화구분.unique()

array(['일반영화', '독립/예술영화'], dtype=object)

In [30]:
m.to_csv("../../data/processed/KOBIS_개봉일람_1.csv", index=False)